# WCRP 2023 - LL02: Open Earth System Science in Cloud Part 2

This part is about plotting data with the DeepESDL platform. There are two main perspectives. First, most data formats from `xarray` can directly be plotted. Additionally, it is possible to setup the xcube viewer and use it as a very intuitive way to explore more complex raster datasets.

First of all let's import all important libraries. This time we will include the Viewer component from xcube. Also the 'matplotlib' is included. This is only necessary if you want to use customizations. In general this is not necessary for simple plots. You can indicate if it is used by the `plt.` in front of functions.

In [ ]:
import xcube
from xcube.core.store import new_data_store
from xcube.webapi.viewer import Viewer
import matplotlib.pyplot as plt

## Data Loading

In this notebook we will only use the Earth system data cube (ESDC) from the public DeepESDL store. Please review for short how to load the data from the last notebook.


In [ ]:
# TODO: define the datastore for accessing public DeepESDL data
#store = 

# TODO: Load the data cube esdc-8d-0.25deg-256x128x128-3.0.1.zarr
#dataset = 


## Plotting with `xarray`

The `xarray` library is capable to plot data. All important libraries for this are already loaded with xcube in the DeepESDL platform. The plots are actually done by the famous `matplotlib` library based on the data. But you can also customize your plots.

### Simple 1D plots

First we want to concentrate on simple line plots and plots of one dimensional data. For this we will have a look at the air temperatures in 2m height in Kigali.
 This is a very simple plot and no customization is done. This technique is great for exploration.

In [ ]:
data = dataset['air_temperature_2m'].sel(
    lat = -1.94995000,
    lon = 30.05885000,
    method='nearest')
data.plot()

The plot function allows a wide variety of inputs to control the graphics. Here, `matplotlib` is an extensive library to explore. The `xarray` package not just delivers the data, but also descriptions about the axes and so the axes labels are correct. Let's customize the plot a bit further:

- colors
- markers
- aspect ratio of the figure.

In [ ]:
data.plot(marker = 'o', color='green')
plt.title('2m air temperature in Kigali')

In [ ]:
data.plot(linestyle='--', marker = 'o', color='green', aspect=2, size=8)
plt.title('2m air temperature in Kigali')

### More 1D plots

There are many more kinds of plots for 1D data. And also there are ways to customize the plots even more. This is all based on the `matplotlib` library. So far we used `plot` without any specialization. We could have used as well: `dataset.plot.line()`. This is what is happening in the background anyway. But there are other possibilities as well.

In [ ]:
data.plot.hist()

Even more intersting is to bundle several plots together. You need to use slicing in more than one variable for this. The next plot shows the 2m air temperature at 3 different positions.

In [ ]:
data = dataset['air_temperature_2m'].sel(
    lat = [10, 0, -10],
    lon = 30,
    method='nearest')
data.plot()

Well this is not what we intended. In this case we need to specify the kind of plot as `line` our variable for the x-axis as well. This should be the `time` variable.
Please remember that those are just number experiments. In notebook 3 we will have a look at a more realistic plotting scenario.

In [ ]:
data = dataset['air_temperature_2m'].sel(
    lat = [10, 0, -10],
    lon = 30,
    method='nearest')
data.plot.line(x="time")

### 2D plots

Two dimensional plots are very famous and with `xarray` they are also easy to create. Let's have a look at the 2m air temperature in Rwanda 5 years ago.

**Attention**
We didn't run into this yet, but it is not possible to use `method=nearest` together with `slice`. If you want to combine both, You need to split the command into two parts.

*Hint:* You can also combine different commands for more efficiency.

In [ ]:
# This will cause an error
#data = dataset['air_temperature_2m'].sel(
#    lat = slice(-2.8, -1.0),
#    lon = slice(28.8, 30.9),
#    time = '2018-10-24',
#    method = 'nearest')

# This is recommended
rwanda_data = dataset['air_temperature_2m'].sel(
    lat = slice(-2.8, -1.0),
    lon = slice(28.8, 30.9)
)
rwanda_data.sel(
    time = '2018-10-24',
    method='nearest'
).plot()
plt.title("2m air temperature of Rwanda, around 2018-10-24")

We want to have a look at one more example. In case values are not available (`missing` or `NaN`), the `plot` function will actually care for this. Here is an example about this. The Gross Primary Productivity (GPP) is only available as data, where landmass is present. Also, there is no GPP in the desert region.

In [ ]:
africa_data = dataset['gross_primary_productivity'].sel(
    lat=slice(-35, 37),
    lon=slice(-18, 52)
)
africa_data.sel(
    time = '2018-10-24',
    method='nearest'
).plot()
plt.title("GPP in Africa around 2018-10-24")

This is also the moment to show how to save such plots. For this you need to have the following command in the same cell as the plotting commands. Here we also introduce the possibility to change the colormap in the plot command.

In [ ]:
africa_data.sel(
    time = '2018-10-24',
    method='nearest'
).plot(
    cmap='viridis'
)
# Save the figure
plt.savefig('GPP_Africa_2018_10_24.pdf')

## xcube viewer

As part of the DeepESDL platform, there is one more feature for plotting. This is the xcube viewer. It is a whole dynamic and interactive plotting experience for datacubes. We want to concentrate our view to the central african region between 2015 and 2020. Therefore we need to create a dataset first.

In [ ]:
# first limit to variables
climate_data = dataset[
['air_temperature_2m', 'evaporation_era5', 'precipitation_era5']
]

# Now limit to spatial dimensions
central_africa_data = climate_data.sel(
    lat = slice(-20, 20),
    lon = slice(-18, 52)
)

# now limit the time frame
final_data = central_africa_data.sel(
    time = slice('2015-01-01', '2015-12-31')
)

This datacube is still to big to find a suitable way for plotting. But the xcube viewer can be a solution.

Please set up a scaling according to the data to get the best experience. You can do this with clicking the colorbar.

In [ ]:
viewer = Viewer()
viewer.add_dataset(final_data)
viewer.show()